Este documento fue realizado traduciendo el documento original : https://www.kaggle.com/ash316/novice-to-grandmaster

# De principiante a  Grandmaster- Que dicen los cientificos de datos?

![](https://www.kaggle.com/static/images/host-home/host-home-recruiting.png?v=daa83a72816a)

Kaggle es la plataforma de Data Science más grande del mundo con más de 1 millón de usuarios, y es una plataforma excelente para que estudiantes como yo aprendan y crezcan en el campo de Data Science y Machine Learning. Cuenta con usuarios de varios dominios, como estadísticos, científicos de datos y practicantes de aprendizaje automático. Este conjunto de datos publicado por Kaggle es una joya para gente como yo, a quienes les gusta analizar e investigar datos. En este cuaderno, trataremos de encontrar algunas tendencias o algunas preguntas comunes, que a cada científico de datos en comun les gustaría saber, como las herramientas más utilizadas, los recursos para aprender ciencia de datos, etc.

El mayor problema que podríamos enfrentar es la falsa y falsa respuesta. Como se trata de una encuesta, no todos responderán con las credenciales adecuadas y, por lo tanto, supongo que habrá muchos más atípicos. Sumergámonos directamente en el conjunto de datos y obtengamos algunos conocimientos ...

# Introduccion

### ¿Quiénes son los científicos de datos??

Un científico de datos es un estadístico o un programador, que limpia, administra y organiza datos, realiza estadísticas y análisis descriptivos para desarrollar ideas, construir modelos predictivos y resolver problemas relacionados con negocios. Veamos que dicen los científicos de datos en kaggle ..

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import base64
import io
from scipy.misc import imread
import codecs
from IPython.display import HTML
from matplotlib_venn import venn2
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
response=pd.read_csv('../input/multipleChoiceResponses.csv',encoding='ISO-8859-1')

In [ ]:
response.head()

## Algunos análisis básicos

In [ ]:
print('El número total de encuestados:',response.shape[0])
print('Número total de países con encuestados:',response['Country'].nunique())
print('País con mayor número de encuestados.:',response['Country'].value_counts().index[0],'con',response['Country'].value_counts().values[0],'encuestados')
print('Respondiente mas joven:',response['Age'].min(),' y el encuestado más viejo:',response['Age'].max())

¿¿Seriamente?? El Respondedor más joven no tiene ni un año de edad. Jaja ¿Y cómo es que el abuelo sigue trabajando a la edad de 100 años? Puede ser una respuesta falsa.

## División de género

In [ ]:
plt.subplots(figsize=(22,12))
sns.countplot(y=response['GenderSelect'],order=response['GenderSelect'].value_counts().index)
plt.show()

La gráfica muestra claramente que hay muchos más encuestados masculinos en comparación con las mujeres. Parece que las damas estaban ocupadas con su trabajo, **o las damas no trabajan** ...: p. Es una broma.

## Los encuestados por país

In [ ]:
resp_coun=response['Country'].value_counts()[:15].to_frame()
sns.barplot(resp_coun['Country'],resp_coun.index,palette='inferno')
plt.title('Top 15 países por número de encuestados')
plt.xlabel('')
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.show()
tree=response['Country'].value_counts().to_frame()
squarify.plot(sizes=tree['Country'].values,label=tree.index,color=sns.color_palette('RdYlGn_r',52))
plt.rcParams.update({'font.size':20})
fig=plt.gcf()
fig.set_size_inches(40,15)
plt.show()

**USA e India**, constituyen máximos encuestados, alrededor de 1/3 del total. Del mismo modo, Chile tiene el número más bajo de encuestados. ¿Este gráfico es suficiente para decir que la mayoría de los usuarios de Kaggle son de India y EE. UU. No lo creo, ya que los usuarios totales en Kaggle son más de 1 millón, mientras que el número de encuestados es de solo 16k.

## Compensación

Los científicos de datos son uno de los individuos mejor pagados. Vamos a ver qué dicen los encuestadores.

In [ ]:
response['CompensationAmount']=response['CompensationAmount'].str.replace(',','')
response['CompensationAmount']=response['CompensationAmount'].str.replace('-','')
rates=pd.read_csv('../input/conversionRates.csv')
rates.drop('Unnamed: 0',axis=1,inplace=True)
salary=response[['CompensationAmount','CompensationCurrency','GenderSelect','Country','CurrentJobTitleSelect']].dropna()
salary=salary.merge(rates,left_on='CompensationCurrency',right_on='originCountry',how='left')
salary['Salary']=pd.to_numeric(salary['CompensationAmount'])*salary['exchangeRate']
print('El salario maximo es USD $',salary['Salary'].dropna().astype(int).max())
print('El salario minimo es USD $',salary['Salary'].dropna().astype(int).min())
print('El salario promedio es USD $',salary['Salary'].dropna().astype(int).median())

¡Mira ese enorme salario! Eso es **incluso más grande que el PIB de muchos países**. Otro ejemplo de respuesta falsa. El salario mínimo tal vez un caso de un estudiante. El salario medio muestra que Data Scientist disfruta de buenos beneficios salariales.

In [ ]:
plt.subplots(figsize=(15,8))
salary=salary[salary['Salary']<1000000]
sns.distplot(salary['Salary'])
plt.title('Distribucion salarial',size=15)
plt.show()

### Compensación por país

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sal_coun=salary.groupby('Country')['Salary'].median().sort_values(ascending=False)[:15].to_frame()
sns.barplot('Salary',sal_coun.index,data=sal_coun,palette='RdYlGn',ax=ax[0])
ax[0].axvline(salary['Salary'].median(),linestyle='dashed')
ax[0].set_title('Países que pagan salarios más altos')
ax[0].set_xlabel('')
max_coun=salary.groupby('Country')['Salary'].median().to_frame()
max_coun=max_coun[max_coun.index.isin(resp_coun.index)]
max_coun.sort_values(by='Salary',ascending=True).plot.barh(width=0.8,ax=ax[1],color=sns.color_palette('RdYlGn'))
ax[1].axvline(salary['Salary'].median(),linestyle='dashed')
ax[1].set_title('Compensación de los 15 principales países encuestados')
ax[1].set_xlabel('')
ax[1].set_ylabel('')
plt.subplots_adjust(wspace=0.8)
plt.show()

El gráfico de la izquierda muestra los 15 principales países con salarios medios altos. Es bueno ver que estos países proporcionan un salario mejor que el salario promedio del conjunto de datos completo. De manera similar, el gráfico de la derecha muestra el salario promedio de los 15 países principales por encuestados. El gráfico más impactante es para **India**. India tiene los 2dos más altos encuestados, pero todavía tiene el salario medio más bajo en el gráfico. Las personas en los EE. UU. Tienen un salario casi 10 veces más que sus contrapartes en la India. ¿Cuál puede ser la razón? ¿Son los profesionales de TI en la India realmente mal pagados? Lo comprobaremos más tarde.

### Salario por género

In [ ]:
plt.subplots(figsize=(10,8))
sns.boxplot(y='GenderSelect',x='Salary',data=salary)
plt.ylabel('')
plt.show()

El salario para los hombres parece ser alto en comparación con otros.

## Edad

In [ ]:
plt.subplots(figsize=(15,8))
response['Age'].hist(bins=50,edgecolor='black')
plt.xticks(list(range(0,80,5)))
plt.title('Distribucion por edad')
plt.show() 

Los encuestados son jóvenes, la mayoría de ellos en el rango de edad entre 25 y 35 años.

## Profesion & Especialidad

In [ ]:
f,ax=plt.subplots(1,2,figsize=(25,15))
sns.countplot(y=response['MajorSelect'],ax=ax[0],order=response['MajorSelect'].value_counts().index)
ax[0].set_title('Especialidad')
ax[0].set_ylabel('')
sns.countplot(y=response['CurrentJobTitleSelect'],ax=ax[1],order=response['CurrentJobTitleSelect'].value_counts().index)
ax[1].set_title('Trabajo actual')
ax[1].set_ylabel('')
plt.subplots_adjust(wspace=0.8)
plt.show()

La ciencia de datos y el aprendizaje automático se utilizan en casi todas las industrias. Esto es evidente en el gráfico de la izquierda, ya que personas de diferentes áreas de interés como Física, Biología, etc. lo están tomando para una mejor comprensión de los datos. El gráfico del lado derecho muestra el trabajo actual de los encuestados. Una gran parte de los encuestados son Dats Scientists. Pero como se trata de datos de encuestas, sabemos que puede haber muchas respuestas ambiguas. Más adelante veremos si estos encuestados son científicos de datos reales o científicos de datos autoproclamados.

## Compensación por título de trabajo

In [ ]:
sal_job=salary.groupby('CurrentJobTitleSelect')['Salary'].median().to_frame().sort_values(by='Salary',ascending=False)
ax=sns.barplot(sal_job.Salary,sal_job.index,palette=sns.color_palette('inferno',20))
plt.title('Compensación por título de trabajo',size=15)
for i, v in enumerate(sal_job.Salary): 
    ax.text(.5, i, v,fontsize=10,color='white',weight='bold')
fig=plt.gcf()
fig.set_size_inches(8,8)
plt.show()

El Practicante de Investigación de Operaciones tiene el salario medio más alto seguido por el Predictive Modeler y el Data Scientist. El informático y los programadores tienen la compensación más baja.

## Machine Learning

In [ ]:
f,ax=plt.subplots(1,2,figsize=(25,12))
skills=response['MLSkillsSelect'].str.split(',')
skills_set=[]
for i in skills.dropna():
    skills_set.extend(i)
plt1=pd.Series(skills_set).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0],plt1.index,ax=ax[0],palette=sns.color_palette('inferno_r',15))
ax[0].set_title('ML Skills')
tech=response['MLTechniquesSelect'].str.split(',')
techniques=[]
for i in tech.dropna():
    techniques.extend(i)
plt1=pd.Series(techniques).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0],plt1.index,ax=ax[1],palette=sns.color_palette('inferno_r',15))
ax[1].set_title('ML Tecnicas usadas')
plt.subplots_adjust(wspace=0.8)
plt.show()

Es evidente que la mayoría de los encuestados están trabajando con Aprendizaje supervisado, y la Regresión logística es la favorita entre ellos. No existe un algoritmo que sea el mejor para todos los dominios de clasificación. Una forma de seleccionar un algoritmo para un dominio en particular es mediante la validación cruzada en los datos de entrenamiento.

In [ ]:
f,ax=plt.subplots(1,2,figsize=(25,12))
ml_nxt=response['MLMethodNextYearSelect'].str.split(',')
nxt_year=[]
for i in ml_nxt.dropna():
    nxt_year.extend(i)
pd.Series(nxt_year).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('winter_r',15),ax=ax[0])
tool=response['MLToolNextYearSelect'].str.split(',')
tool_nxt=[]
for i in tool.dropna():
    tool_nxt.extend(i)
pd.Series(tool_nxt).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('winter_r',15),ax=ax[1])
plt.subplots_adjust(wspace=0.8)
ax[0].set_title('ML Métodos el año que viene')
ax[1].set_title('ML Herramienta el año que viene')
plt.show()

Es evidente que el próximo año va a ver un aumento en el número de practicantes de **Deep Learning**. El aprendizaje profundo y las redes neuronales o, en definitiva, la IA es un tema candente para el próximo año. También en términos de herramientas, Python se prefiere más a R. Las herramientas de Big Data como Spark y Hadoop también tienen una buena participación en los próximos años.

## Las mejores plataformas para aprender

In [ ]:
plt.subplots(figsize=(6,8))
learn=response['LearningPlatformSelect'].str.split(',')
platform=[]
for i in learn.dropna():
    platform.extend(i)
pd.Series(platform).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('winter',15))
plt.title('Las mejores plataformas para aprender',size=15)
plt.show()

Mi favorito es Kaggle, es la fuente más buscada para aprender Data Science, ya que tiene notebooks de Data Scientists realmente experimentados. La siguiente opción son los cursos en línea, es decir, los MOOC. Plataformas como coursera, udacity proporcionan videos interactivos y ejercicios para el aprendizaje. De manera similar, los canales de Youtube como Siraj Raval y otros ofrecen un medio gratuito para estudiar. Todos estos medios están por encima de los libros de texto. La razón quizás sea que los libros de texto a menudo tienen un contenido limitado, o que las personas les gusta más mirar videos y aprender.

## Hardware utilizado

In [ ]:
plt.subplots(figsize=(10,10))
hard=response['HardwarePersonalProjectsSelect'].str.split(',')
hardware=[]
for i in hard.dropna():
    hardware.extend(i)
pd.Series(hardware).value_counts().sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('inferno',10))
plt.title('Maquinas usadas')
plt.show()

Dado que la mayoría de los encuestados se encuentran en la categoría de edad inferior a 25, que es donde la mayoría de los estudiantes se clasifican, una computadora portátil básica es la máquina más utilizada para el trabajo.

## De dónde obtengo conjuntos de datos??


In [ ]:
plt.subplots(figsize=(15,15))
data=response['PublicDatasetsSelect'].str.split(',')
dataset=[]
for i in data.dropna():
    dataset.extend(i)
pd.Series(dataset).value_counts().plot.pie(autopct='%1.1f%%',colors=sns.color_palette('Paired',10),startangle=90,wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'white' })
plt.title('Origen de Datasets')
my_circle=plt.Circle( (0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')
plt.show()

Con cientos de conjuntos de datos disponibles, Kaggle es la fuente más buscada para datasets.

## Código compartido

In [ ]:
plt.subplots(figsize=(15,15))
code=response['WorkCodeSharing'].str.split(',')
code_share=[]
for i in code.dropna():
    code_share.extend(i)
pd.Series(code_share).value_counts().plot.pie(autopct='%1.1f%%',shadow=True,colors=sns.color_palette('Set3',10),startangle=90,wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'white' })
plt.title('medio de codigo compartido')
my_circle=plt.Circle( (0,0), 0.65, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')
plt.show()

Github es la plataforma más utilizada para compartir código y proyectos. Las ventajas de usar github son:

1) Control de versiones de tus proyectos.

2) Explore los proyectos de otros en GitHub, obtenga un código inspirado más o contribuya a su proyecto.

3) Colabore con otros, permitiendo que otras personas contribuyan a sus proyectos o usted contribuya a otros proyectos, y muchos más.

## Desafíos en la ciencia de datos

In [ ]:
plt.subplots(figsize=(15,18))
challenge=response['WorkChallengesSelect'].str.split(',')
challenges=[]
for i in challenge.dropna():
    challenges.extend(i)
plt1=pd.Series(challenges).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0],plt1.index,palette=sns.color_palette('inferno',25))
plt.title('Desafíos en la ciencia de datos')
plt.show()

El principal desafío en Data Science es **obtener la información adecuada**. El gráfico muestra claramente que los datos sucios son el desafío mayor. Ahora, ¿qué son los datos sucios? Los datos sucios son un registro de base de datos que contiene errores. Los datos sucios pueden ser causados por una serie de factores que incluyen registros duplicados, datos incompletos o desactualizados y el análisis incorrecto de campos de registros de sistemas dispares. Afortunadamente, los conjuntos de datos de Kaggle están bastante limpios y estandarizados.

Algunos otros desafíos importantes son la **falta de ciencia de datos y talento para el aprendizaje automático, dificultad para obtener datos y falta de herramientas**. Es por eso que la ciencia de datos es el trabajo más sexy en el siglo XXI. Con la creciente cantidad de datos, esta demanda crecerá sustancialmente.

## Satisfacción laboral

In [ ]:
satisfy=response.copy()
satisfy['JobSatisfaction'].replace({'10 - Highly Satisfied':'10','1 - Highly Dissatisfied':'1','I prefer not to share':np.NaN},inplace=True)
satisfy.dropna(subset=['JobSatisfaction'],inplace=True)
satisfy['JobSatisfaction']=satisfy['JobSatisfaction'].astype(int)
satisfy_job=satisfy.groupby(['CurrentJobTitleSelect'])['JobSatisfaction'].mean().sort_values(ascending=False).to_frame()
ax=sns.barplot(y=satisfy_job.index,x=satisfy_job.JobSatisfaction,palette=sns.color_palette('inferno',20))
fig=plt.gcf()
fig.set_size_inches(8,10)
for i, v in enumerate(satisfy_job.JobSatisfaction): 
    ax.text(.1, i, v,fontsize=10,color='white',weight='bold')
plt.title('top 10 de Satisfacción laboral')
plt.show()

Los científicos de datos y los ingenieros de Aprendizaje automático son las personas más satisfechas (quienes no estarán contentos con tanto dinero), mientras que los programadores tienen la satisfacción laboral más baja. Pero lo que hay que notar aquí es que incluso si los Computer Scientist  tienen un salario más bajo que los programadores, también tienen un buen nivel de satisfacción laboral que los programadores. Así, el salario no es el único criterio o la satisfacción laboral.

## Satisfacción laboral por país

In [ ]:
satisfy=response.copy()
satisfy['JobSatisfaction'].replace({'10 - Highly Satisfied':'10','1 - Highly Dissatisfied':'1','I prefer not to share':np.NaN},inplace=True)
satisfy.dropna(subset=['JobSatisfaction'],inplace=True)
satisfy['JobSatisfaction']=satisfy['JobSatisfaction'].astype(int)
satisfy_job=satisfy.groupby(['Country'])['JobSatisfaction'].mean().sort_values(ascending=True).to_frame()
data = [ dict(
        type = 'choropleth',
        autocolorscale = False,
        colorscale = 'Viridis',
        reversescale = True,
        showscale = True,
        locations = satisfy_job.index,
        z = satisfy_job['JobSatisfaction'],
        locationmode = 'country names',
        text = satisfy_job['JobSatisfaction'],
        marker = dict(
            line = dict(color = 'rgb(200,200,200)', width = 0.5)),
            colorbar = dict(autotick = True, tickprefix = '', 
            title = 'Satisfaction')
            )
       ]

layout = dict(
    title = 'Satisfacción laboral por país',
    geo = dict(
        showframe = True,
        showocean = True,
        oceancolor = 'rgb(0,0,255)',
        projection = dict(
        type = 'chloropleth',
            
        ),
        lonaxis =  dict(
                showgrid = False,
                gridcolor = 'rgb(102, 102, 102)'
            ),
        lataxis = dict(
                showgrid = False,
                gridcolor = 'rgb(102, 102, 102)'
                )
            ),
        )
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='worldmap2010')

El nivel promedio de satisfacción laboral está entre **6-7.5** para la mayoría de los países. Es más bajo en Japón (donde la gente trabaja durante aproximadamente 14 horas) y China. Es más alto en países como Suecia y México.


# Python vs R o (Batman vs Superman)

![](https://blog.webhose.io/wp-content/uploads/2017/08/python-versus-R.png)'

Python y R son los lenguajes de código abierto más utilizados para la ciencia de la información y el aprendizaje automático. Para un científico o analista de datos en ciernes, la duda más grande y complicada es: **¿Con qué lenguaje debería comenzar?** Si bien ambos lenguajes tienen sus propias ventajas y desventajas, depende del propósito del individuo al seleccionar un lenguaje suyo. Su elección. Ambos lenguajes satisfacen las necesidades de diferentes tipos de trabajo. Python es un lenguaje de propósito general, por lo tanto, la integración de la aplicación y la web es más fácil, mientras que R está destinado solo para fines estadísticos y analíticos. El área donde R vencerá completamente a Python son las visualizaciones con la ayuda de paquetes como **ggplot2 y shiny**. Pero Python tiene una mano superior en materia de aprendizaje automático. Así que veamos lo que dicen los encuestadores.

In [ ]:
resp=response.dropna(subset=['WorkToolsSelect'])
resp=resp.merge(rates,left_on='CompensationCurrency',right_on='originCountry',how='left')
python=resp[(resp['WorkToolsSelect'].str.contains('Python'))&(~resp['WorkToolsSelect'].str.contains('R'))]
R=resp[(~resp['WorkToolsSelect'].str.contains('Python'))&(resp['WorkToolsSelect'].str.contains('R'))]
both=resp[(resp['WorkToolsSelect'].str.contains('Python'))&(resp['WorkToolsSelect'].str.contains('R'))]

### Lenguaje recomendado para principiantes

In [ ]:
response['LanguageRecommendationSelect'].value_counts()[:2].plot.bar()
plt.show()

Claramente Python es el lenguaje recomendado para principiantes. La razón de esto tal vez se deba a su sintaxis similar a la del inglés y su funcionalidad de propósito general.

## Recomendación por usuarios de Python y R

In [ ]:
labels1=python['LanguageRecommendationSelect'].value_counts()[:5].index
sizes1=python['LanguageRecommendationSelect'].value_counts()[:5].values

labels2=R['LanguageRecommendationSelect'].value_counts()[:5].index
sizes2=R['LanguageRecommendationSelect'].value_counts()[:5].values


fig = {
  "data": [
    {
      "values": sizes1,
      "labels": labels1,
      "domain": {"x": [0, .48]},
      "name": "Language",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },     
    {
      "values": sizes2 ,
      "labels": labels2,
      
      "textposition":"inside",
      "domain": {"x": [.54, 1]},
      "name": "Language",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Language Recommended By Python and R users",
        "annotations": [
            {
                "font": {
                    "size": 30
                },
                "showarrow": False,
                "text": "Python",
                "x": 0.17,
                "y": 0.5
            },
            {
                "font": {
                    "size": 30
                },
                "showarrow": False,
                "text": "R",
                "x": 0.79,
                "y": 0.5}]}}
py.iplot(fig, filename='donut')

Este es un hallazgo interesante. Cerca de **91.6%** de los usuarios de Python recomiendan Python como el primer lenguaje para los principiantes, mientras que solo **67.2%** Los usuarios de R recomiendan R como el primer lenguaje. También **20.6%** Los usuarios de R recomiendan Python pero solo **1.68%** Los usuarios de Python recomiendan R como primer lenguaje. Una cosa a tener en cuenta es que los usuarios de ambos recomiendan los mismos lenguajes, es decir, SQL, Matlab y C / C ++. Solo he considerado los 5 mejores lenguajes recomendados, por lo que el porcentaje cambiará si los consideramos todos. Pero la diferencia sería de solo el 2-3%.

### Necesario o No??

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
response['JobSkillImportancePython'].value_counts().plot.pie(ax=ax[0],autopct='%1.1f%%',explode=[0.1,0,0],shadow=True,colors=['g','lightblue','r'])
ax[0].set_title('Necesidad de Python')
ax[0].set_ylabel('')
response['JobSkillImportanceR'].value_counts().plot.pie(ax=ax[1],autopct='%1.1f%%',explode=[0,0.1,0],shadow=True,colors=['lightblue','g','r'])
ax[1].set_title('Necesidad de R')
ax[1].set_ylabel('')
plt.show()

Claramente, Python es una habilidad mucho más necesaria en comparación con R.

Gracias especiales a [Steve Broll](https://www.kaggle.com/stevebroll) por ayudar en la combinación de colores.

### Número de usuarios por lenguajes

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
pd.Series([python.shape[0],R.shape[0],both.shape[0]],index=['Python','R','Both']).plot.bar(ax=ax[0])
ax[0].set_title('Numero de usuarios')
venn2(subsets = (python.shape[0],R.shape[0],both.shape[0]), set_labels = ('Python Users', 'R Users'))
plt.title('Diagrama de Venn de usuarios')
plt.show()

El número de usuarios de Python es definitivamente más que los usuarios de R. Esto puede deberse a la curva de aprendizaje fácil de Python. Sin embargo hay más usuarios que conocen ambos lenguajes. Estas respuestas pueden ser de científicos de datos establecidos, ya que tienden a tener conocimientos en varios lenguajes y herramientas.

## Compensation

In [ ]:
py_sal=(pd.to_numeric(python['CompensationAmount'].dropna())*python['exchangeRate']).dropna()
py_sal=py_sal[py_sal<1000000]
R_sal=(pd.to_numeric(R['CompensationAmount'].dropna())*R['exchangeRate']).dropna()
R_sal=R_sal[R_sal<1000000]
both_sal=(pd.to_numeric(both['CompensationAmount'].dropna())*both['exchangeRate']).dropna()
both_sal=both_sal[both_sal<1000000]
trying=pd.DataFrame([py_sal,R_sal,both_sal])
trying=trying.transpose()
trying.columns=['Python','R','Both']
print('Promedio salarial para individuos que usan Python:',trying['Python'].median())
print('Promedio salarial para individuos que usan R:',trying['R'].median())
print('Promedio salarial para individuos que usan ambos lenguajes:',trying['Both'].median())


In [ ]:
trying.plot.box()
plt.title('Compensación por lenguaje')
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.show()

Los codificadores de Python tienen un salario medio ligeramente más alto en comparación con sus homólogos de R. Sin embargo, las personas que conocen ambos idiomas tienen un salario medio bastante alto en comparación con los dos.

## Languaje utilizado por profesionales

In [ ]:
py1=python.copy()
r=R.copy()
py1['WorkToolsSelect']='Python'
r['WorkToolsSelect']='R'
r_vs_py=pd.concat([py1,r])
r_vs_py=r_vs_py.groupby(['CurrentJobTitleSelect','WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('CurrentJobTitleSelect','WorkToolsSelect','Age').plot.barh(width=0.8)
fig=plt.gcf()
fig.set_size_inches(10,15)
plt.title('Cargos vs Lenguajes usados',size=15)
plt.show()

Como mencioné anteriormente, R vence a Python en visualizacion. Por lo tanto, las personas con títulos de trabajo como analista de datos, analista de negocios donde los gráficos y visuales desempeñan un papel muy importante, prefieren R sobre Python. De manera similar, casi el 90% de los estadísticos usan R. También como se indicó anteriormente, Python es mejor en materia de Aprendizaje automático, por lo que los ingenieros de Aprendizaje automático, científicos de datos y otros como DBA o programadores prefieren Python sobre R.

Por lo tanto, para datos visuales ---> R de lo contrario ----> Python.

**Nota: esta gráfica no es para el lenguaje recomendado por los profesionales, sino para las herramientas utilizadas por los profesionales.**

## Función en el trabajo vs lenguaje

In [ ]:
r_vs_py=pd.concat([py1,r])
r_vs_py=r_vs_py.groupby(['JobFunctionSelect','WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('JobFunctionSelect','WorkToolsSelect','Age').plot.barh(width=0.8)
fig=plt.gcf()
fig.set_size_inches(10,15)
plt.title('Función en el trabajo vs lenguaje usado')
plt.show()

Como ya mencioné, **R sobresale en analítica, pero Python vence en Aprendizaje automático.** La gráfica muestra que R tiene influencia cuando se trata de analítica pura, pero otras formas en que Python gana.

## Permanencia Vs lenguaje usado

In [ ]:
r_vs_py=pd.concat([py1,r])
r_vs_py=r_vs_py.groupby(['Tenure','WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('Tenure','WorkToolsSelect','Age').plot.barh(width=0.8)
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.title('Permanencia en el trabajo vs lenguaje usado')
plt.show()

Como habíamos visto anteriormente, Python es muy recomendable para principiantes. Por lo tanto, la proporción de usuarios de Python es mayor en los años iniciales de codificación. Sin embargo, la brecha entre los lenguajes se reduce con los años, a medida que aumenta la experiencia de codificación.

## Industria vs lenguaje utilizado

In [ ]:
r_vs_py=pd.concat([py1,r])
r_vs_py=r_vs_py.groupby(['EmployerIndustry','WorkToolsSelect'])['Age'].count().to_frame().reset_index()
r_vs_py.pivot('EmployerIndustry','WorkToolsSelect','Age').plot.barh(width=0.8)
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.title('Industria vs lenguaje utilizado')
plt.show()

R vence a Python en las industrias gubernamentales, de seguros y sin fines de lucro. Del mismo modo, Python vence a R con un margen muy grande en la industria de tecnología y militar. En el resto de otras industrias, la proporción de Python parece ser aproximadamente un 15-20% más que la de R.

## Herramientas comunes con Python y R

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,15))
py_comp=python['WorkToolsSelect'].str.split(',')
py_comp1=[]
for i in py_comp:
    py_comp1.extend(i)
plt1=pd.Series(py_comp1).value_counts()[1:15].sort_values(ascending=False).to_frame()
sns.barplot(plt1[0],plt1.index,ax=ax[0],palette=sns.color_palette('inferno_r',15))
R_comp=R['WorkToolsSelect'].str.split(',')
R_comp1=[]
for i in R_comp:
    R_comp1.extend(i)
plt1=pd.Series(R_comp1).value_counts()[1:15].sort_values(ascending=False).to_frame()
sns.barplot(plt1[0],plt1.index,ax=ax[1],palette=sns.color_palette('inferno_r',15))
ax[0].set_title('Herramientas comunmente usadas con Python')
ax[1].set_title('Herramientas comunmente usadas con R')
plt.subplots_adjust(wspace=0.8)
plt.show()

**SQL** parece ser la herramienta complementaria más común utilizada con ambos idiomas. SQL es el lenguaje principal para consultar grandes bases de datos, por lo que saberlo bien es una gran ventaja.

# Preguntando a los científicos de datos



![](https://ewebdesign.com/wp-content/uploads/2017/01/zarget_banner2.gif)

Si escribo con éxito un programa Hello World, ¿eso me hace programador o desarrollador? Si derroto a mis amigos en una carrera, ¿eso me convierte en la persona más veloz del mundo? La respuesta es bastante obvia **NO.** Este es el problema con las personas emergentes de informática y TI. Basados en sus habilidades y experiencia limitadas, comienzan a considerarse mucho más de lo que realmente son. Muchos de ellos comienzan a llamarlos Practicantes de Aprendizaje Automático, incluso si realmente no han trabajado en proyectos de la vida real y solo han trabajado en algunos conjuntos de datos simples. Similarmente, muchas respuestas aquí deben ser una respuesta falsa. Vamos a comprobar cuántos de ellos son los verdaderos profesionales de la ciencia de datos.

In [ ]:
response['DataScienceIdentitySelect'].value_counts()

Entonces, aproximadamente el 26% del total de los encuestados se consideran a sí mismos como científicos de datos. ¿Qué significa algo de esto? ¿Siguen aprendiendo o están desempleados? Por ahora vamos a considerarlos como un No.

## Títulos de trabajo actuales

In [ ]:
plt.subplots(figsize=(10,8))
scientist=response[response['DataScienceIdentitySelect']=='Yes']
scientist['CurrentJobTitleSelect'].value_counts().sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('inferno',15))
plt.title('Títulos de trabajo',size=15)
plt.show()

Sorprendentemente no hay **ninguna entrada para el científico de datos del título del trabajo**. Esto podría deberse a que las personas con CurrentJobTitleSelect como Data Scientist (que podría estar trabajando como Data Scientist) podrían no haber respondido la pregunta: **"¿Actualmente te consideras un Data Scientist?"**

Hay muchas habilidades comunes y superpuestas entre los trabajos como analista de datos, científico de datos y expertos en aprendizaje automático, estadísticos, etc. Por lo tanto, ellos también tienen habilidades similares y se consideran a sí mismos como científicos de datos, aunque no estén etiquetados de la misma manera. Ahora vamos a comprobar si la suposición anterior era verdadera.

In [ ]:
true=response[response['CurrentJobTitleSelect']=='Data Scientist']

Fue de hecho **Verdad**. Las personas con su CurrentJobTitle como Data Scientist no respondieron la pregunta **"¿Actualmente te consideras un Data Scientist?"**. Así que también los considero científicos reales de datos.

In [ ]:
scientist=pd.concat([scientist,true])
scientist['CurrentJobTitleSelect'].shape[0]

Entonces, del total de encuestados, alrededor del **40%** de ellos son científicos de datos o tienen habilidades para el mismo.

## Pasion y Division de DS

In [ ]:
plt.subplots(figsize=(10,8))
coun=scientist['Country'].value_counts()[:15].sort_values(ascending=False).to_frame()
sns.barplot(coun.Country,coun.index,palette='inferno')
plt.title('Países por número de científicos de datos',size=15)
plt.show()

El gráfico es similar al gráfico demográfico en el que mostramos el número de usuarios por país. La diferencia es que los números se han reducido ya que solo hemos considerado científicos de datos.

## Situación laboral y educación

In [ ]:
f,ax=plt.subplots(1,2,figsize=(25,10))
sns.countplot(y=scientist['EmploymentStatus'],ax=ax[0])
ax[0].set_title('Estatus laboral')
ax[0].set_ylabel('')
sns.countplot(y=scientist['FormalEducation'],order=scientist['FormalEducation'].value_counts().index,ax=ax[1],palette=sns.color_palette('viridis_r',15))
ax[1].set_title('Educacion formal')
ax[1].set_ylabel('')
plt.subplots_adjust(wspace=0.8)
plt.show()

About **67%** of the data scientists are employed full-time, while about **11-12%** of them are unemployed but looking for job. On the education side it is evident that about **45-46%** of the data scientists hold a **master's degree**, while about **23-24%** of them have a **bachelor's degree or a doctoral degree**. Thus education seems to be an important factor for becoming a data scientist. Let's see how does the salary vary according to the education.

Alrededor de **67%** de los datos se emplean a tiempo completo, mientras que alrededor de **11-12%** de ellos están desempleados pero buscando trabajo. Por el lado de la educación, es evidente que alrededor del **45-46%** de los científicos de datos tienen un **grado de maestría**, mientras que alrededor del **23-24%** de ellos tienen un **título de licenciatura o un doctorado**. Por lo tanto, la educación parece ser un factor importante para convertirse en un científico de datos. Veamos cómo varía el salario según la educación.

## Compensación por la educación formal


In [ ]:
plt.subplots(figsize=(25,12))
comp_edu=scientist.merge(salary,left_index=True,right_index=True,how='left')
comp_edu=comp_edu[['FormalEducation','Salary']]
sns.boxplot(x='FormalEducation',y='Salary',data=comp_edu)
plt.title('Compensation vs Educacion')
plt.xticks(rotation=90)
plt.show()

Esto es sorprendente, ya que los rangos salariales para licenciatura, maestría y doctorado parecen muy similares. La mediana de la licenciatura parece ser un poco alta en comparación con la maestría y el doctorado. No esperaba esto ya que muchos de los científicos de datos tienen una maestría. Pero creo que **Experiencia laboral** es más importante que cualquier título. Tal vez los titulares de la licenciatura tienen más experiencia en comparación con los otros dos.

## Trabajo previo y cambio salarial

In [ ]:
f,ax=plt.subplots(1,2,figsize=(30,15))
past=scientist['PastJobTitlesSelect'].str.split(',')
past_job=[]
for i in past.dropna():
    past_job.extend(i)
pd.Series(past_job).value_counts().sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('summer',25),ax=ax[0])
ax[0].set_title('Trabajo previo')
sal=scientist['SalaryChange'].str.split(',')
sal_change=[]
for i in sal.dropna():
    sal_change.extend(i)
pd.Series(sal_change).value_counts().sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('summer',10),ax=ax[1])
ax[1].set_title('Cambio salarial')
plt.subplots_adjust(wspace=0.9)
plt.show()

Es evidente que la mayoría de las personas que se cambian a Data Science obtienen un aumento salarial de **6-20% o más**. Con esta creciente demanda de Data Scientists, el salario también puede aumentar con el tiempo.

## Herramientas utilizadas en el trabajo

In [ ]:
plt.subplots(figsize=(8,8))
tools=scientist['WorkToolsSelect'].str.split(',')
tools_work=[]
for i in tools.dropna():
    tools_work.extend(i)
pd.Series(tools_work).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('RdYlGn',15))
plt.show()

Observaciones similares, Python, R y SQL son las herramientas o lenguajes más utilizados en Data Science

## ¿De dónde aprendieron ellos???

In [ ]:
course=scientist['CoursePlatformSelect'].str.split(',')
course_plat=[]
for i in course.dropna():
    course_plat.extend(i)
course_plat=pd.Series(course_plat).value_counts()
blogs=scientist['BlogsPodcastsNewslettersSelect'].str.split(',')
blogs_fam=[]
for i in blogs.dropna():
    blogs_fam.extend(i)
blogs_fam=pd.Series(blogs_fam).value_counts()
labels1=course_plat.index
sizes1=course_plat.values

labels2=blogs_fam[:5].index
sizes2=blogs_fam[:5].values


fig = {
  "data": [
    {
      "values": sizes1,
      "labels": labels1,
      "domain": {"x": [0, .48]},
      "name": "MOOC",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },     
    {
      "values": sizes2 ,
      "labels": labels2,
      
      "textposition":"inside",
      "domain": {"x": [.54, 1]},
      "name": "Blog",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Blogs and Online Platforms",
        "showlegend":True,
        "annotations": [
            {
                "font": {
                    "size": 12
                },
                "showarrow": False,
                "text": "MOOC's",
                "x": 0.18,
                "y": 0.5
            },
            {
                "font": {
                    "size": 12
                },
                "showarrow": False,
                "text": "BLOGS",
                "x": 0.83,
                "y": 0.5}]}}
py.iplot(fig, filename='donut')

Coursera es la plataforma más favorecida por los científicos de datos para el aprendizaje de la ciencia de datos. Mi voto personal también va para Coursera, donde puedes aprender cosas desde cero hasta avanzadas en la misma plataforma. No se limita a un solo lenguaje como Python o R, sino que también tiene cursos que cubren otros lenguaje como Scala, etc. Del mismo modo KDNuggets es el blog más preferido.

## Tiempo empleado en tareas

Un científico de datos no siempre está construyendo modelos predictivos, también es responsable de la calidad de los datos, la recopilación de los datos correctos, análisis, etc. Veamos cuánto tiempo pasa un científico de datos en estas diferentes tareas.

In [ ]:
import itertools
plt.subplots(figsize=(22,10))
time_spent=['TimeFindingInsights','TimeVisualizing','TimeGatheringData','TimeModelBuilding']
length=len(time_spent)
for i,j in itertools.zip_longest(time_spent,range(length)):
    plt.subplot((length/2),2,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.5)
    scientist[i].hist(bins=10,edgecolor='black')
    plt.axvline(scientist[i].mean(),linestyle='dashed',color='r')
    plt.title(i,size=20)
    plt.xlabel('% Time')
plt.show()

La línea punteada es la línea media.
Vamos a hacerlo paso a paso:

  - **TimeGatheringData:** Es, sin duda, la parte más lenta. Obtener los datos es la tarea más minuciosa de todo el proceso, a la que sigue la Limpieza de datos (que no se muestra como datos no disponibles), que es otro proceso que consume mucho tiempo. Por lo tanto, la recopilación de datos correctos y la limpieza de los datos son el proceso más lento.
  
  - **TimeVisualizing:** Es probablemente el proceso que consume menos tiempo (y probablemente el más agradable ...: p), y se reduce aún más si usamos Enterprise Tools como Tableau, Qlik, Tibco, etc., lo que ayuda a Construyendo gráficos y tableros con características simples de arrastrar y soltar.
  
  - **TimeFindingInsights:** Se sigue después de visualizar los datos, lo que implica encontrar datos y patrones en los datos, dividirlos y cortarlos en trozos para encontrar información sobre los procesos de negocios. Parece un poco más lento que el de TimeVisualizing.
  
  - **TimeModelBuilding:** Es donde los científicos de datos construyen modelos predictivos, sintonizan estos modelos, etc. Es el segundo proceso que consume más tiempo después de TimeDataGathering.


## Servicios en la nube

Con el aumento del tamaño de los datos, no es posible procesar los datos y realizar análisis predictivos en las infraestructuras del servidor físico. Por lo tanto, la nube lleva el análisis predictivo a un nivel superior, con escalabilidad su principal ventaja. Gestionan un servicio que le permite construir fácilmente modelos de aprendizaje automático que funcionan con cualquier tipo de datos, de cualquier tamaño. Permite comprobar cuáles son las plataformas en la nube más utilizadas por los científicos de datos.

In [ ]:
cloud=['WorkToolsFrequencyAmazonML','WorkToolsFrequencyAWS','WorkToolsFrequencyCloudera','WorkToolsFrequencyHadoop','WorkToolsFrequencyAzure']
plt.subplots(figsize=(30,15))
length=len(cloud)
for i,j in itertools.zip_longest(cloud,range(length)):
    plt.subplot((length/2+1),3,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.5)
    sns.countplot(i,data=scientist)
    plt.title(i,size=20)
    plt.ylabel('')
    plt.xlabel('')
plt.show()

It is evident that **AmazonAWS**, which is a public cloud service provider is the most used cloud platform, followed by Hadoop. Hadoop is an open-source software framework used for distributed storage and processing of dataset of big data. For reading more about Hadoop, **<a href='https://www.analyticsvidhya.com/blog/2014/05/hadoop-simplified/'>Check this</a>**

Es evidente que **AmazonAWS**, que es un proveedor de servicios de nube pública, es la plataforma de nube más utilizada, seguida por Hadoop. Hadoop es un marco de software de código abierto utilizado para el almacenamiento distribuido y el procesamiento de conjuntos de datos de big data. Para leer más sobre Hadoop, **<a href='https://www.analyticsvidhya.com/blog/2014/05/hadoop-simplified/'>Revise esto</a>**

## Importancia de las visualizaciones

In [ ]:
f,ax=plt.subplots(1,2,figsize=(25,12))
sns.countplot(scientist['JobSkillImportanceVisualizations'],ax=ax[0])
ax[0].set_title('Importancia laboral para visuales')
ax[0].set_xlabel('')
scientist['WorkDataVisualizations'].value_counts().plot.pie(autopct='%2.0f%%',colors=sns.color_palette('Paired',10),ax=ax[1])
ax[1].set_title('Uso de las visualizaciones en proyectos.')
my_circle=plt.Circle( (0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')
plt.show()

Las visualizaciones son una parte muy integral de los proyectos de Data Science, y el gráfico anterior también muestra lo mismo. Casi todos los proyectos de ciencia de datos, es decir, el **99%** de los proyectos tienen visualizaciones, no importa lo grande o pequeño que sea. Aproximadamente **95%** de los científicos de datos dicen que las habilidades de visualización son agradables de tener o necesarias. Las visuales ayudan a comprender y comprender los datos con mayor rapidez, no solo para los profesionales, sino también para los clientes objetivo, que pueden no ser técnicamente expertos.

## Herramientas de BI

El software de inteligencia empresarial es un tipo de software de aplicación diseñado para recuperar, analizar, transformar e informar datos para inteligencia empresarial. Hacen que la visualización y el análisis de datos sean muy simples en comparación con la forma de codificación normal en Python o R. El único inconveniente es que son **exclusivos y costosos**. Permite comprobar cuáles son las herramientas de BI empresariales más utilizadas por los científicos de datos.

In [ ]:
BI=['WorkToolsFrequencyQlik','WorkToolsFrequencySAPBusinessObjects','WorkToolsFrequencyTableau','WorkToolsFrequencyTIBCO','WorkToolsFrequencyAngoss','WorkToolsFrequencyIBMCognos','WorkToolsFrequencyKNIMECommercial','WorkToolsFrequencyExcel']
plt.subplots(figsize=(30,25))
length=len(BI)
for i,j in itertools.zip_longest(BI,range(length)):
    plt.subplot((length/2),3,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.5)
    sns.countplot(i,data=scientist)
    plt.title(i,size=20)
    plt.ylabel('')
    plt.xlabel('')
plt.show()

Había leído o escuchado en alguna parte que **Excel es el pan y la mantequilla para los analistas**. Esto sigue siendo cierto, ya que Excel sigue siendo popular entre los científicos de datos. Sin embargo, la herramienta de BI utilizada con más frecuencia parece ser **Tableau**. Personalmente uso Tableau y es bastante fácil de usar, arrastre y suelte y ya tiene sus gráficos listos.

## Conocimiento de los algoritmos (Matemáticas y Estadísticas)

In [ ]:
f,ax=plt.subplots(1,2,figsize=(25,12))
sns.countplot(y=scientist['AlgorithmUnderstandingLevel'],order=scientist['AlgorithmUnderstandingLevel'].value_counts().index,ax=ax[0],palette=sns.color_palette('summer',15))
sns.countplot(scientist['JobSkillImportanceStats'],ax=ax[1])
ax[0].set_title('Comprensión del algoritmo')
ax[0].set_ylabel('')
ax[1].set_title('Conocimiento de las estadísticas')
ax[1].set_xlabel('')
plt.show()

Los científicos de datos tienen un buen conocimiento de conceptos matemáticos como Estadística y Álgebra Lineal, que son la parte más importante de los algoritmos de aprendizaje automático. Pero, ¿es esta matemática realmente necesaria, ya que muchas bibliotecas estándar como scikit, tensorflow, keras, etc. ya tienen todas estas cosas implementadas? Pero los científicos de datos experimentados dicen que deberíamos tener una buena comprensión de las matemáticas detrás de los algoritmos. Alrededor de **95%** de los científicos de datos dicen que las estadísticas son un activo importante en Data Science

## Utilidad de la plataforma de aprendizaje

In [ ]:
plt.subplots(figsize=(25,35))
useful=['LearningPlatformUsefulnessBlogs','LearningPlatformUsefulnessCollege','LearningPlatformUsefulnessCompany','LearningPlatformUsefulnessKaggle','LearningPlatformUsefulnessCourses','LearningPlatformUsefulnessProjects','LearningPlatformUsefulnessTextbook','LearningPlatformUsefulnessYouTube']
length=len(useful)
for i,j in itertools.zip_longest(useful,range(length)):
    plt.subplot((length/2),3,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.2)
    scientist[i].value_counts().plot.pie(autopct='%1.1f%%',colors=['g','lightblue','r'],wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'white' })
    plt.title(i,size=25)
    my_circle=plt.Circle( (0,0), 0.7, color='white')
    p=plt.gcf()
    p.gca().add_artist(my_circle)
    plt.xlabel('')
    plt.ylabel('')
plt.show()

Los gráficos de anillos anteriores muestran la opinión de los científicos de datos sobre las distintas plataformas para aprender la ciencia de datos. La trama se ve mejor para **Proyectos**, donde el porcentaje no es útil casi **0%**. Según mi opinión personal, los proyectos son la mejor plataforma o forma de aprender algo en la industria de TI. Las otras plataformas excelentes son **Cursos en línea y Kaggle**. Los gráficos para otras plataformas son bastante similares entre sí.

## ¿Qué debe tener el curriculum vitae?

In [ ]:
f,ax=plt.subplots(1,2,figsize=(22,8))
sns.countplot(y=scientist['ProveKnowledgeSelect'],order=scientist['ProveKnowledgeSelect'].value_counts().index,ax=ax[0],palette=sns.color_palette('inferno',15))
ax[0].set_title('Como probar mi conocimiento.')
sns.countplot(scientist['JobSkillImportanceKaggleRanking'],ax=ax[1])
ax[1].set_title('Rango de Kaggle')
plt.show()

Es evidente que la experiencia laboral en proyectos ML y en las competiciones Kaggle refleja el conocimiento de Data Science. También un rango de Kaggle puede ser algo bueno en el currículum. Como mencioné anteriormente, la experiencia laboral relevante podría tener un valor mayor en comparación con cualquier título de maestría o doctorado. Por lo tanto, esta afirmación es válida, ya que los científicos de datos prefieren la experiencia laboral sobre el grado, como se ve en el gráfico anterior.


## ¿Cómo buscaron empleo?

In [ ]:
plt.subplots(figsize=(10,8))
scientist.groupby(['EmployerSearchMethod'])['Age'].count().sort_values(ascending=True).plot.barh(width=0.8,color=sns.color_palette('winter',10))
plt.title('Metodos de busqueda de empleo',size=15)
plt.ylabel('')
plt.show()

Muchos científicos de datos conocen los trabajos a través de sus amigos o familiares o fueron contactados directamente por la compañía. Por lo tanto, debemos mantener adecuadamente nuestros perfiles profesionales como Linkedin y seguir actualizándolos, ya que dichos sitios de redes podrían ayudarlo a conseguir el trabajo de sus sueños.

## Comprobando las respuestas libres

Este archivo contiene las respuestas de forma libre respondidas por los encuestados. El problema con este es que al ser una respuesta de forma libre, cada usuario responderá a su manera. Lo que quiero decir es que tendremos diferentes respuestas para la misma cosa. Un ejemplo de esto que observé es la biblioteca **Pandas está escrito como pandas, Pandas, panda y en muchas formas diferentes.** Por lo tanto, trataré de analizar este archivo utilizando **nltk (Natural Language Toolkit).**

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
free=pd.read_csv('../input/freeformResponses.csv')
stop_words=set(stopwords.words('english'))
stop_words.update(',',';','!','?','.','(',')','$','#','+',':','...')

## Motivación detrás de trabajar en Kaggle

In [ ]:
kaggle=b'...'

In [ ]:
motivation=free['KaggleMotivationFreeForm'].dropna().apply(nltk.word_tokenize)
motivate=[]
for i in motivation:
    motivate.extend(i)
motivate=pd.Series(motivate)
motivate=([i for i in motivate.str.lower() if i not in stop_words])
f1=open("kaggle.png", "wb")
f1.write(codecs.decode(kaggle,'base64'))
f1.close()
img1 = imread("kaggle.png")
hcmask1 = img1
wc = WordCloud(background_color="black", max_words=4000, mask=hcmask1, 
               stopwords=STOPWORDS, max_font_size= 60,width=1000,height=1000)
wc.generate(" ".join(motivate))
plt.imshow(wc)
plt.axis('off')
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.show()

### Vamos a flexibilizar nuestra fuerza en la ciencia de datos ...

El wordcloud muestra la motivación de los usuarios para trabajar en kaggle. Claramente, el aprendizaje de la ciencia de datos, el aprendizaje automático, el interés en el mismo, la curiosidad, la diversión y la recuperación de conjuntos de datos son algunos de los más relevantes.

## Bibliotecas más utilizadas

In [ ]:
library=free['WorkLibrariesFreeForm'].dropna().apply(nltk.word_tokenize)
lib=[]
for i in library:
    lib.extend(i)
lib=pd.Series(lib)
lib=([i for i in lib.str.lower() if i not in stop_words])
lib=pd.Series(lib)
lib=lib.value_counts().reset_index()
lib.loc[lib['index'].str.contains('Pandas|pandas|panda'),'index']='Pandas'
lib.loc[lib['index'].str.contains('Tensorflow|tensorflow|tf|tensor'),'index']='Tensorflow'
lib.loc[lib['index'].str.contains('Scikit|scikit|sklearn'),'index']='Sklearn'
lib=lib.groupby('index')[0].sum().sort_values(ascending=False).to_frame()
R_packages=['dplyr','tidyr','ggplot2','caret','randomforest','shiny','R markdown','ggmap','leaflet','ggvis','stringr','tidyverse','plotly']
Py_packages=['Pandas','Tensorflow','Sklearn','matplotlib','numpy','scipy','seaborn','keras','xgboost','nltk','plotly']
f,ax=plt.subplots(1,2,figsize=(18,10))
lib[lib.index.isin(Py_packages)].sort_values(by=0,ascending=True).plot.barh(ax=ax[0],width=0.9,color=sns.color_palette('viridis',15))
ax[0].set_title('Bibliotecas Py más utilizadas')
lib[lib.index.isin(R_packages)].sort_values(by=0,ascending=True).plot.barh(ax=ax[1],width=0.9,color=sns.color_palette('viridis',15))
ax[1].set_title('Bibliotecas R más utilizadas')
ax[1].set_ylabel('')
plt.show()

Una breve información sobre las bibliotecas:

### Python:

1) **Sklearn** - Para algoritmos de aprendizaje automático. Esta biblioteca tiene casi todos los algoritmos importantes de aprendizaje automático utilizados para las industrias.

2) **Pandas, Matlotlib y Seaborn** Generalmente se usan juntos para el trabajo de análisis y visualización.

3) **TensorFlow y Keras** Usado para Deep Learning

4) **Numpy and Scipy** Usado para cálculos científicos.

5) **nltk** Se utiliza para el procesamiento de lenguaje natural.

### R:

1) **dply** dplyr es el paquete para la manipulación rápida de datos.

2) **ggplot2 and shiny** El famoso paquete de R para hacer hermosos gráficos. Los efectos visuales de Python no se parecen en nada a la calidad de los elementos visuales creados con esta biblioteca.

3) **Caret and randomforest** Para fines de aprendizaje automático.

4) **tidyr** Herramientas para cambiar el diseño de sus conjuntos de datos.

5) **stringr** Herramientas fáciles de aprender para expresiones regulares y cadenas de caracteres.

**Los prospectos (folium en Python) y Plotly son bibliotecas comunes en ambos idiomas, y se utilizan para crear gráficos interactivos como mapas geográficos, etc.**

# Conclusiones
Algunas breves ideas que recogimos del Notebook:

1) La mayoría de los encuestados son de EE. UU. Seguidos de India. EE. UU. También tenía el número máximo de científicos de datos seguidos por India. También el salario medio es el más alto en Estados Unidos.

2) La mayoría de los encuestados está en el rango de edad 20-35, lo que demuestra que la ciencia de los datos es bastante famosa en los jóvenes.

3) Los encuestados no solo se limitan a la especialización en Ciencias de la Computación, sino también a carreras como Estadísticas, ciencias de la salud, etc., que muestran que la Ciencia de Datos es un dominio interdisciplinario.

4) La mayoría de los encuestados están totalmente empleados.

5) Kaggle, Cursos en línea (Coursera, eDx, etc.), Proyectos y Blogs (KDNuggets, AnalyticsVidya, etc.) son los principales recursos / plataformas para aprender Data Science.

6) Kaggle tiene la parte más alta para la adquisición de datos, mientras que Github tiene la parte más alta para compartir el código.

7) Los científicos de datos tienen el nivel más alto de satisfacción laboral y el segundo salario medio más alto (después del analista de investigación de operaciones). Por el contrario, los programadores tienen el menor nivel de satisfacción laboral y uno de los salarios menos medianos también.

8) Los científicos de datos también obtienen un aumento de alrededor del 6-20% de sus trabajos anteriores.

#### Consejos para científicos de datos 

1) Aprenda **Python, R y SQL** ya que son los lenguajes más utilizados por los científicos de datos. Python y R ayudarán en el análisis analítico y en el modelado predictivo, mientras que SQL es el mejor para consultar las bases de datos.

2) Aprenda técnicas de aprendizaje automático como **Regresión logística, Árboles de decisión, Máquinas de vectores de soporte**, etc., ya que son las técnicas / algoritmos de aprendizaje automático más utilizados.

3) **Deep Learning y redes neuronales** serán las técnicas más buscadas en el futuro, por lo que un buen conocimiento de ellas será muy útil.

4) Desarrollar habilidades para **Recolectar datos** y **Limpiar los datos**, ya que son los procesos que más tiempo consumen en el flujo de trabajo de un científico de datos.

5) **Las visualizaciones** son muy importantes en los proyectos de Data Science y casi todos los proyectos requieren visualizaciones para comprender mejor los datos. Por lo tanto, uno debe aprender la visualización de datos, ya que los científicos de datos consideran que es  **necesario o mejor tener habilidades.**

6) **Matemáticas y estadísticas** son muy importantes en Data Science, por lo que deberíamos entenderlo bien para entender cómo funciona el algoritmo.

7) **Los proyectos** son la mejor manera de aprender Data Science de acuerdo con Data Scientists. Por lo tanto, trabajar en proyectos te ayudará a aprender mejor sobre Data Science.

8) **La experiencia con proyecto ML en empresas y competiciones en Kaggle** son las mejores maneras de mostrar tus conocimientos de trabajo en Data Science. Trabajar en proyectos de ML en una empresa brinda la experiencia de trabajar con conjuntos de datos del mundo real, mejorando así el conocimiento. Las competiciones Kaggle también son un gran medio, ya que competirás con los científicos de datos en todo el mundo. También un * Kaggle Rank** puede ser un buen USP en el currículum.


Así que quisiera concluir mi análisis aquí. Muchas gracias por echar un vistazo a este cuaderno. He vinculado otro cuaderno que trata con datos de encuestas. Échale un vistazo.

Enlace: [HackerRank Survey](https://www.kaggle.com/ash316/coders-not-hackers-hackerrank)

Espero que a todos les haya gustado el cuaderno. Cualquier sugerencia y comentarios son siempre bienvenidos.

### Por favor vota por este cuaderno, ya que me anima a hacerlo mejor.


![](http://68.media.tumblr.com/e1aed171ded2bd78cc8dc0e73b594eaf/tumblr_o17frv0cdu1u9u459o1_500.gif)

Disclaimer: Este cuaderno fue traducido por mi persona, para apoyar el libre acceso a la informacion, no se intento realizar una copia o plagio del mismo.